# Introducción a la recuperación de información
Autor: Eric S. Tellez <eric.tellez@infotec.mx><br/>

La recuperación de información se crea a partir de la necesidad de simplificar el acceso y revisión de documentos en grandes colecciones. Estas colecciones pueden ser homogeneas o heterogeneas, tanto en su contenido como en su formato. En su inicio, se consideraban colecciones unicamente de texto pero las necesidades de información se han diversificado, y ahora es común encontrar sistemas de recuperación de información sobre otros datos como imagenes o videos, o include multimodales, esto es que puedan usar diferentes tipos de objetos. Un objeto puede ser un documento de texto, una imagen, o cualquier otro tipo de datos que se desee tener acceso. 

Mantener un sistema de información homogeneo puede simplificar su mantenimiento enormemente, por lo que si es posible, se puede intentar mantener cierta homegeneidad. Para sistemas de fuentes abiertas como puede ser la web, esto será posible.

En general, se puede ver un sistema de recuperación de información en tres grandes partes:

1. Recolección de datos, normalización y modelado matemático.
2. Indexamiento, interpretación de consultas y optimización
3. Agregación y filtrado de resultados, presentación de los mismos


En este curso se visitarán parcialmente todas estas partes. En general se usaran conjuntos de datos previamente recolecados, aunque se invita a usar sus propias colecciones de tal forma que puedan sacar provecho de los temas de manera más amplia. La normalización puede ir desde el simple preprocesamiento de los datos hasta manipulaciones y transformaciones dependientes del dominio. Una vez aplicado el modelado adecuado a las colecciones, las representaciones matemáticas suelen ser vectores de alta dimensión para cada objeto.

En cuanto al indexamiento se utilizarán dos tipos de algoritmos, búsqueda mediante índices invertidos y búsqueda por índices métricos. Ambas tienen sus nichos de aplicación y usarlas adecuadamente requiere conocer los problemas y sus modelados.

La presentación de los resultados puede ser tan simple como una lista de resultados más relevantes y una pequeña muestra del objeto, o más complejo que requiera alguna técnica de visualización. Todo esto dependerá del dominio de aplicación y la naturaleza del sistema de recuperación de información.


# Ejemplos de sistemas de recuperación información

- Google,scholar google, google images, google news, youtube...
- Bing!
- Yahoo search
- Yandex
- DuckDuckGo
- ...

También es común que el sistema de recuperación de información no sea el principal producto, si no más bien un complemento imperdible
como es el caso de los sistemas de streaming de videos, películas, música, redes sociales, etc.

# Ejemplos de implementación de search engines open source
- [Apache Lucene](https://lucene.apache.org/)
- [Apache SOLR](https://solr.apache.org/)
- [ElasticSearch](https://www.elastic.co/es/what-is/elasticsearch)
- [hnswlib](https://github.com/nmslib/hnswlib)
- [FAISS](https://github.com/facebookresearch/faiss)
- [TextSearch.jl](https://github.com/sadit/TextSearch.jl)
- [InvertedFiles.jl](https://github.com/sadit/InvertedFiles.jl)
- [SimilaritySearch.jl](https://github.com/sadit/SimilaritySearch.jl)


# Ejemplo


Construcción de un índice de búsqueda para una colección de mensajes de Twitter. Los mensajes se pueden ver como documentos cortos. Se preprocesan y se modalen en vectores mediante un vocabulario. Después se construye el índice con dichos vectores y se hace posible la búsqueda eficiente. Note que existen etiquetas que se pueden usar para otras tareas; de manera más genérica, se pueden ver como metadatos asociados a los elementos del corpus.

Nota: La primera vez que se corra este notebook los paquetes necesarios se instalaran de manera automática, es posible que tome un tiempo considerable.



In [1]:
using Pkg
Pkg.activate(".")

using TextSearch, InvertedFiles, SimilaritySearch, TextSearch, CodecZlib, JSON, LinearAlgebra, HypertextLiteral

using Downloads: download

  Activating project at `~/IR-2022/Unidades`


## Funciones para leer y modelar el corpus

In [2]:
function parse_corpus(corpusfile)
    corpus, labels = String[], String[]
    open(corpusfile) do f
        for line in eachline(GzipDecompressorStream(f))
            r = JSON.parse(line)
            push!(labels, r["klass"])
            push!(corpus, r["text"])    
        end
    end
    
    corpus, labels
end

function text_model_and_vectors(corpus)
    textconfig = TextConfig(group_usr=true, group_url=true, del_diac=true, lc=true, group_num=true, nlist=[1], qlist=[])
    model = VectorModel(IdfWeighting(), TfWeighting(), textconfig, corpus)    
    vectors = vectorize_corpus(model, textconfig, corpus)
    for v in vectors
        normalize!(v)
    end

    (; textconfig, model, vectors)
end

function create_dataset(corpusfile)
    corpus, labels = parse_corpus(corpusfile)
    (; labels, corpus, text_model_and_vectors(corpus)...)
end

create_dataset (generic function with 1 method)

In [3]:
display(@htl "<h1>Cargando el corpus</h1>")
dbname = "emo50k.json.gz"
dbfile = "../data/$dbname"
baseurl = "https://github.com/sadit/TextClassificationTutorial/blob/main/data"

!isfile(dbfile) && download("$baseurl/$dbname?raw=true", dbfile)
D = create_dataset(dbfile)

@show unique(D.labels), D.model

<h1>Cargando el corpus</h1>

(unique(D.labels), D.model) = (["😰", "😥", "😊", "😏", "♡", "💔", "🙂", "😋", "😌", "🌚", "👌", "😪", "😤", "🙃", "🤤", "😴", "😢", "😅", "😑", "😠", "😂", "😜", "🤓", "💙", "😀", "🤗", "🤣", "😒", "✨", "😐", "😞", "😁", "😱", "👏", "😫", "😍", "❤", "😣", "🙊", "🙏", "🙄", "🤭", "💜", "🤔", "😬", "👀", "😉", "😈", "😡", "😳", "🙈", "😻", "😔", "😓", "💕", "🎶", "😭", "😕", "♥", "💖", "😎", "😘", "😃", "😩"], {VectorModel global_weighting=IdfWeighting(), local_weighting=TfWeighting(), train-voc=45374, train-n=50000, maxoccs=93991})


(["😰", "😥", "😊", "😏", "♡", "💔", "🙂", "😋", "😌", "🌚"  …  "💕", "🎶", "😭", "😕", "♥", "💖", "😎", "😘", "😃", "😩"], {VectorModel global_weighting=IdfWeighting(), local_weighting=TfWeighting(), train-voc=45374, train-n=50000, maxoccs=93991})

### Note que las etiquetas son emojis, que hay 50k ejemplos y un vocabulario de más de 45k tokens. A continuación se construirá el índice invertido usando el paquete <https://github.com/sadit/TextSearch.jl>, claramente, el código solo funcionará con dicha implementación, pero el procedimiento general aplicaría a otras implementaciones.

In [4]:
invfile = WeightedInvertedFile(length(D.model.voc))
append!(invfile, VectorDatabase(D.vectors))

{WeightedInvertedFile vocsize=45374, n=50000}

In [5]:
function search_and_display(invfile, D, q, k)
    res = KnnResult(k)
    @time search(invfile, vectorize(D.model, D.textconfig, q), res)
    L = []
    for (i, (id, dist)) in enumerate(res)
        dist = round(dist, digits=2)
        push!(L, @htl """<tr><td>$i</td> <td>$id => $dist</td> <td>$(D.labels[id])</td> <td>$(D.corpus[id])</td> </tr>""")
       # println(i, " ", id => round(dist, digits=2), " ", D.labels[i], " ", D.corpus[id])
    end

    display(@htl """
    <h3>Resultados para la consulta "$q"</h3>
    <table>
    $L
    </table>
    """)
end

search_and_display (generic function with 1 method)

In [6]:
search_and_display(invfile, D, "no importa distancia", 5)
search_and_display(invfile, D, "cuando nos vemos?", 5)
search_and_display(invfile, D, "feliz cumpleaños?", 3)
search_and_display(invfile, D, "ola k ase", 3)
search_and_display(invfile, D, "mi humilde opinión", 10)

  0.000325 seconds (18 allocations: 3.344 KiB)


1,39550 => 0.03,💕,No importa la distancia _emo _usr _url
2,3870 => 0.38,😓,Me caga la distancia _emo
3,11115 => 0.52,😃,No importa cuando... _emo _emo _url
4,38044 => 0.57,😥,"tiempo y distancia, quizás sea eso... _emo"
5,12728 => 0.57,😑,_usr A ti no te importa _emo


  0.000257 seconds (19 allocations: 3.359 KiB)


1,41790 => 0.22,😂,Nos vemos!! _emo _emo _emo _emo _url
2,27642 => 0.3,😜,_usr En la tarde nos vemos _emo _emo _emo
3,5656 => 0.32,😻,Nos vemos pronto _usr _usr _emo _emo _url
4,2835 => 0.43,😘,_usr Muchas gracias! Nos vemos pronto _emo
5,44970 => 0.43,😍,_usr Pronto nos vemos hermana _emo _emo


  0.000143 seconds (18 allocations: 3.328 KiB)


1,37575 => 0.09,❤,_usr Feliz Cumpleaños _emo
2,41849 => 0.09,😁,Feliz Cumpleaños _emo _emo _usr
3,12769 => 0.09,🙊,Feliz Cumpleaños _usr _emo _emo _emo _emo _emo


  0.000017 seconds (17 allocations: 3.172 KiB)


1,23340 => 0.64,😎,"_usr _usr Siempre, todo y todos los días de la semana desde ase muchoooooossss años _emo _emo"
2,6952 => 0.7,😤,_usr k bueno k tu mizmo lo reconozcAszzzz _emo
3,1874 => 0.72,😃,"Despertares!! (By OLA) # _emo _emo en Distrito Federal, Mexico _url"


  0.000188 seconds (17 allocations: 3.266 KiB)


1,19966 => 0.32,🙏,"Bueno, cada quien su vida, ya se, solo es una humilde opinión !! _emo _emo _emo"
2,7756 => 0.5,😍,_usr es la persona más humilde que conozco _emo
3,4755 => 0.62,🤔,_usr Bajo tu opinión fue o no falta? _emo
4,24948 => 0.62,😊,Gracias por compartir la información y tu opinión _emo _url
5,20021 => 0.64,😉,Cuantos Likes para mi humilde oficina _usr _emo _emo @ CocoBongo _url
6,34164 => 0.66,🤗,"Para ser feliz, no escuches la opinión de los demás _emo"
7,47417 => 0.68,👏,_usr Totalmente de acuerdo contigo en todo. Comparto tu opinión _emo _emo _emo _emo
8,846 => 0.69,😌,Cambie de opinión quiero seguir siendo rubia _emo _emo _emo _emo _emo _emo
9,15711 => 0.72,😌,"Tu opinión no me interesa más. No tengo nada que demostrarte, y si piensas lo peor de mi, MEJOR! _emo"
10,9033 => 0.72,😑,"Lo siento pero mi tez humilde no me deja ser guapo y estar bueno, eso sin mencionar la rodilla que me chingue! _emo _emo"


### También es posible consultar elementos del mismo corpus, lo cual permité inspeccionar la estructura interna de la colección. Esto será de utilidad más adelante en el curso.

In [7]:
for i in 1:10
    qID = rand(1:length(D.corpus))
    search_and_display(invfile, D, D.corpus[qID], 7)
end

  0.001288 seconds (43 allocations: 5.016 KiB)


1,25424 => -0.0,🤔,Bills vs Jets... _emo no pueden perder los dos? _emo _emo _emo _emo _emo
2,14016 => 0.62,😳,No pueden ir los dos _emo
3,23327 => 0.72,😬,_usr Ganara Jets por un Gol de campo _emo
4,778 => 0.73,🙃,_usr De los dos _emo
5,36818 => 0.74,😡,No tengo miedo a perder _emo
6,44664 => 0.74,😋,Pueden ser dos si si?? _emo
7,48226 => 0.74,🙄,_usr los dos! _emo


  0.005938 seconds (49 allocations: 6.625 KiB)


1,22509 => -0.0,😎,Que se agarren las _usr mi estimado David si entramos como octavo a liguilla _emo _emo _emo _url
2,33967 => 0.71,😜,_usr jajajjajaja que se agarren _emo
3,34332 => 0.72,😞,Adiós liguilla _emo _emo
4,40481 => 0.73,😻,me encanta que me agarren el pelo _emo
5,25965 => 0.73,🙈,Y así queremos liguilla _emo _emo _emo _emo _emo _emo _emo _emo
6,14822 => 0.75,😉,_usr _usr _usr Solo que se aplique en la liguilla _emo _emo _emo
7,15561 => 0.75,🤤,HABEMUS LIGUILLA _emo _emo _emo _emo


  0.002510 seconds (29 allocations: 4.047 KiB)


...

1,8494 => 0.0,😠,Pinches ratas me tienen hasta la madre _emo _emo _emo _emo _emo _emo _emo
2,8813 => 0.41,😡,_usr _usr Pinches ratas de mierda! _emo_usr _usr _usr _usr
3,37926 => 0.6,😤,Estoy hasta la madre de todo _emo _emo _emo
4,5248 => 0.6,😠,_usr siiii par ratas!!! _emo
5,37737 => 0.63,😤,Ya estoy hasta la madre de esto _emo
6,12270 => 0.64,😰,En la madre _emo _url
7,16631 => 0.65,😍,Mi madre la madre de todo mexico la madre del mundo _emo _emo _emo _url


  0.000823 seconds (29 allocations: 4.078 KiB)


1,1563 => 0.0,😪,Necesito un llavero para las llaves del carro _emo
2,154 => 0.67,😤,"No jodas, neta como necesito un carro _emo"
3,222 => 0.69,😩,Necesito _emo _emo _emo _emo _emo _emo _emo _emo _emo _emo
4,35530 => 0.7,🙂,Mi mama me escondió las llaves desde ayer _emo
5,41844 => 0.71,😢,"Pues ya no me fui a Cancún, ya encontramos las llaves. _emo _emo"
6,47466 => 0.72,😞,Te necesito _emo
7,11272 => 0.73,😞,Lo necesito _emo


  0.005221 seconds (47 allocations: 5.344 KiB)


1,18635 => 0.0,❤,JAJAJAJAJA que puto asco pero a la vez está cute hacer pendejadas nacas en familia _emo _url
2,1316 => 0.67,✨,el lugar más cute _emo _url
3,37843 => 0.69,😠,Que asco de persona _emo _url
4,22647 => 0.7,😎,_usr Puto _emo
5,35578 => 0.71,🌚,_usr Puto. _emo
6,752 => 0.73,👀,JAJAJAJAJA _emo _url
7,10613 => 0.73,😌,Jajajajaja _emo _url


# Actividades

- Revisión de los servicios y empresas mencionados 
- Revisión de los proyectos open source mencionados
- Cargue este notebook y modifique las consultas (necesita Jupyter y el kernel de Julia <https://github.com/JuliaLang/IJulia.jl>)
- Revise la bibliografía sobre recuperación de información, en especial [^SMR2008] se encuentra libre desde <https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf>

# Bibliografía
- [SMR2008] Schütze, H., Manning, C. D., & Raghavan, P. (2008). Introduction to information retrieval (Vol. 39, pp. 234-65). Cambridge: Cambridge University Press.
- [BYN1999] Baeza-Yates, R., & Ribeiro-Neto, B. (1999). Modern information retrieval (Vol. 463). New York: ACM press.